# Add XML nodes for austraits data

We will download data from [AusTraits](https://austraits.org/) ([pre-print](https://www.biorxiv.org/content/10.1101/2021.01.04.425314v1)) and add nodes for each species.

Let's start loading the libraries

In [1]:
from pathlib import Path
import os
import json
import urllib
from zipfile import ZipFile
import pandas as pd
import xml.etree.cElementTree as ET
import numpy as np
from xml.dom import minidom

## Species list from NSW

The previous version of the CAPS (whatever that means) table was available from this (non-permanent?) url:

In [2]:
CAPSurl = 'https://www.environment.nsw.gov.au/resources/wildlifelicences/CAPS.xls'
#CAPS = pd.read_excel(CAPSurl, index_col=0)

But we got an updated version from a BioNET export prepared by Renee:

In [3]:
repodir = Path("../") 
inputdir = repodir / "data/"
CAPS = pd.read_excel(inputdir / 'vis-survey-datasheet-6000.PowerQuery.20210708.xlsx')

In [4]:
CAPS

,speciesID,taxonRank,classID,class,orderID,order,familyID,family,sortOrder,genusID,...,stateConservation,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA
0,2358,Species,38,Flora,137,Flora,155,Brassicaceae,7878,3019.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
1,2359,Species,38,Flora,137,Flora,156,Myrtaceae,13245,2638.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
2,2360,Species,38,Flora,137,Flora,156,Myrtaceae,13447,3165.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
3,2361,Species,38,Flora,137,Flora,157,Apiaceae,6031,1823.0,...,Not Listed,True,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
4,2362,Species,38,Flora,137,Flora,157,Apiaceae,6046,1944.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,24289,Species,38,Flora,137,Flora,170,Orchidaceae,14157,3583.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
15728,24291,Species,38,Flora,137,Flora,203,Araliaceae,6378,3931.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False
15729,24292,Species,38,Flora,137,Flora,214,Pittosporaceae,14710,3474.0,...,Critically Endangered,False,Not Sensitive,20378.0,Not Listed,NaN,NaN,False,False,False
15730,24293,Species,38,Flora,137,Flora,206,Malvaceae,12327,1825.0,...,Not Listed,False,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False


In [5]:
CAPS.columns

Index(['speciesID', 'taxonRank', 'classID', 'class', 'orderID', 'order',
       'familyID', 'family', 'sortOrder', 'genusID', 'genus',
       'parentSpeciesID', 'specificEpithet', 'infraspecificEpithet',
       'scientificNameAuthorship', 'scientificNameID', 'speciesCode_Synonym',
       'scientificName', 'scientificNameHTML', 'vernacularName',
       'otherVernacularNames', 'taxonID', 'currentScientificNameCode',
       'currentScientificName', 'currentVernacularName', 'isCurrent',
       'generalTypeID', 'generalType', 'establishmentMeans',
       'primaryGrowthForm', 'primaryGrowthFormGroup', 'secondaryGrowthForms',
       'secondaryGrowthFormGroups', 'stateConservation', 'protectedInNSW',
       'sensitivityClass', 'TSProfileID', 'countryConservation',
       'highThreatWeed', 'widelyCultivatedNativeSpecies', 'CAMBA', 'JAMBA',
       'ROKAMBA'],
      dtype='object')

In [6]:
target = CAPS[CAPS['scientificName'] == "Actinotus helianthi"] 
target

,speciesID,taxonRank,classID,class,orderID,order,familyID,family,sortOrder,genusID,...,stateConservation,protectedInNSW,sensitivityClass,TSProfileID,countryConservation,highThreatWeed,widelyCultivatedNativeSpecies,CAMBA,JAMBA,ROKAMBA
3,2361,Species,38,Flora,137,Flora,157,Apiaceae,6031,1823.0,...,Not Listed,True,Not Sensitive,NaN,Not Listed,NaN,NaN,False,False,False


## Read _austraits_ data 
We will download the file from the [Zenodo repository](https://zenodo.org/record/5112001) using the API url and saving this under the data folder.

In [7]:
repodir = Path("../") 
dataset = "https://zenodo.org/api/records/3568417"
outputdir = repodir / "data/austraits/"

if not os.path.isdir(outputdir):
    os.makedirs(outputdir)

We use urllib to open the url and read the data (if successfully connected!)

In [8]:
def getResponse(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
       data = operUrl.read()
    else:
       print("Error receiving data", operUrl.getcode())
    return data
zrecord = getResponse(dataset)

Response data is in json format, need to parse it:

In [9]:
jsonData = json.loads(zrecord)

In [10]:
jsonData

{'conceptdoi': '10.5281/zenodo.3568417',
 'conceptrecid': '3568417',
 'created': '2021-07-18T06:32:30.575319+00:00',
 'doi': '10.5281/zenodo.5112001',
 'files': [{'bucket': '9c997956-8254-4fcc-a17b-5fe1fd079022',
   'checksum': 'md5:cd7ba1c395b976a02fd4c3c772d88d78',
   'key': 'austraits-3.0.2.rds',
   'links': {'self': 'https://zenodo.org/api/files/9c997956-8254-4fcc-a17b-5fe1fd079022/austraits-3.0.2.rds'},
   'size': 12325324,
   'type': 'rds'},
  {'bucket': '9c997956-8254-4fcc-a17b-5fe1fd079022',
   'checksum': 'md5:ed44176eb71466fe9a4ca1773d6b5961',
   'key': 'austraits-3.0.2.zip',
   'links': {'self': 'https://zenodo.org/api/files/9c997956-8254-4fcc-a17b-5fe1fd079022/austraits-3.0.2.zip'},
   'size': 14738862,
   'type': 'zip'},
  {'bucket': '9c997956-8254-4fcc-a17b-5fe1fd079022',
   'checksum': 'md5:7047ae5b30b1727140000a4daa484722',
   'key': 'dictionary.html',
   'links': {'self': 'https://zenodo.org/api/files/9c997956-8254-4fcc-a17b-5fe1fd079022/dictionary.html'},
   'size': 1

The json data includes a list of files:

In [11]:
for files in jsonData['files']:
    print(files['key'])


austraits-3.0.2.rds
austraits-3.0.2.zip
dictionary.html
NEWS.md
readme.txt


We want to download the zip file with the csv_files

In [12]:
outputfile = outputdir / jsonData['files'][1]['key']

if os.path.isfile(outputfile):
    print('File exists')
else:
    resp = getResponse(jsonData['files'][1]['links']['self'])
    output = open(outputfile,'wb')
    output.write(resp)
    output.close()

File exists


We will read from the zipfile the data that we need:

In [13]:
zfobj = ZipFile(outputfile)
zfobj.namelist()

['austraits-3.0.2/',
 'austraits-3.0.2/taxa.csv',
 'austraits-3.0.2/methods.csv',
 'austraits-3.0.2/definitions.yml',
 'austraits-3.0.2/build_info.md',
 'austraits-3.0.2/contributors.csv',
 'austraits-3.0.2/contexts.csv',
 'austraits-3.0.2/excluded_data.csv',
 'austraits-3.0.2/traits.csv',
 'austraits-3.0.2/taxonomic_updates.csv',
 'austraits-3.0.2/sites.csv',
 'austraits-3.0.2/sources.bib']

### Definitions


In [14]:
import yaml

with zfobj.open('austraits-3.0.2/definitions.yml') as file:
    try:
        ATdefinitions = yaml.safe_load(file)   
        print(ATdefinitions.keys())
    except yaml.YAMLError as exc:
        print(exc)

dict_keys(['traits', 'value_type', 'austraits', 'metadata'])


In [15]:
ATdefinitions['traits']['elements'].keys()

dict_keys(['accessory_cost_mass', 'accessory_cost_fraction', 'bark_delta13C', 'bark_delta15N', 'bark_mass_area', 'bark_density', 'bark_C_per_dry_mass', 'bark_N_per_dry_mass', 'bark_P_per_dry_mass', 'bark_K_per_dry_mass', 'bark_Al_per_dry_mass', 'bark_B_per_dry_mass', 'bark_Ca_per_dry_mass', 'bark_Cu_per_dry_mass', 'bark_Fe_per_dry_mass', 'bark_Mg_per_dry_mass', 'bark_Mn_per_dry_mass', 'bark_Na_per_dry_mass', 'bark_S_per_dry_mass', 'bark_Zn_per_dry_mass', 'bark_morphology', 'bark_thickness', 'bark_ash_content_per_dry_mass', 'bark_cellulose_per_dry_mass', 'bark_lignin_per_dry_mass', 'bark_tannin_per_dry_mass', 'branch_mass_fraction', 'sprout_depth', 'calcicole_status', 'carotenoid_per_area', 'carotenoid_per_dry_mass', 'cell_cross-sectional_area', 'cell_epidermis_Ca_per_fresh_mass', 'cell_hypodermis_Ca_per_fresh_mass', 'cell_internal_parenchyma_Ca_per_fresh_mass', 'cell_palisade_mesophyll_Ca_per_fresh_mass', 'cell_sclerenchyma_Ca_per_fresh_mass', 'cell_spongy_mesophyll_Ca_per_fresh_mass',

In [16]:
print(ATdefinitions['traits']['elements']['fire_response'].keys())

dict_keys(['description', 'type', 'label', 'values'])


In [17]:
print(ATdefinitions['traits']['elements']['fire_response']['values'])

{'fire_killed': 'Plants killed by hot fires', 'resprouts': "Plants resprout from underground storage organ following fire. (For studies that don't differentiate between respouting strength)", 'not_fire_killed_does_not_resprout': 'Plants that are rarely killed by a moderate-intensity fire, but do not resprout', 'fire_not_relevant': 'Plant never affected by fire (for aquatic taxon)', 'unknown': 'Fire status assessed, but unknown'}


### References

In [18]:
from pybtex.database.input import bibtex
parser = bibtex.Parser()

bib_data = parser.parse_bytes(zfobj.open('austraits-3.0.2/sources.bib').read())


In [19]:
#bib_data.entries.keys()
#for key in bib_data.entries.keys():
#    print(key)
bib_data.entries['Adams_1984']
# dir(bib_data.entries['Adams_1984'])

Entry('article',
  fields=[
    ('year', '1984'), 
    ('journal', 'Australian Journal of Botany'), 
    ('title', '{Role of Acacia spp. in nutrient balance and cycling in regenerating Eucalyptus regnans F. Muell. forests. I. Temporal changes in biomass and nutrient content}'), 
    ('volume', '32'), 
    ('number', '2'), 
    ('pages', '205--215'), 
    ('doi', '10.1071/bt9840205')],
  persons=OrderedCaseInsensitiveDict([('author', [Person('Adams, M. A.'), Person('M, P.'), Person('{Attiwill}')])]))

In [52]:
print(bib_data.entries['Adams_1984'].fields['year'])
##''.join(person for person in bib_data.entries['Adams_1984'].persons['author'])
for person in bib_data.entries['Adams_1984'].persons['author']:
    print(person)

dir(person)
person.__str__()

1984
Adams, M. A.
M, P.
{Attiwill}


'{Attiwill}'

In [53]:
import io
import pybtex.database.input.bibtex
import pybtex.plugin
style = pybtex.plugin.find_plugin('pybtex.style.formatting', 'plain')()
citation = style.format_entries(bib_data.entries['Adams_1984'])
print(citation)
#pybtex_html_backend = pybtex.plugin.find_plugin('pybtex.backends', 'html')()
#pybtex_backend = pybtex.plugin.find_plugin('pybtex.backends', 'text')()
#output = io.StringIO()
#pybtex_backend.write_to_stream(citation, output)
#print(output.getvalue())


<generator object BaseStyle.format_entries at 0x11a989af0>


### Traits data

In [22]:
ATtraits = pd.read_csv(zfobj.open('austraits-3.0.2/traits.csv'),low_memory=False)

In [23]:
sppname='Actinotus helianthi'
traits=('fire_response_juvenile','fire_response','fire_cued_seeding')

ss = (ATtraits['taxon_name']==sppname) & (ATtraits['trait_name'].isin(traits))
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
217795,Falster_2005_2,Actinotus helianthi,Myall_Lakes,NaN,Falster_2005_2_05,fire_response,fire_killed,NaN,2002-09,expert_mean,NaN,Actinotus helianthi
454654,NSWFRD_2014,Actinotus helianthi,NaN,NaN,NSWFRD_2014_0177,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Actinotus helianthi


### Taxonomic data

We will read this into a pandas data frame:

In [24]:
df = pd.read_csv(zfobj.open('austraits-3.0.2/taxa.csv'))

Check information from one species

In [25]:
target = df[df['taxon_name'] == "Actinotus helianthi"] 
target

,taxon_name,source,acceptedNameUsageID,scientificNameAuthorship,taxonRank,taxonomicStatus,family,taxonDistribution,ccAttributionIRI,genus
1665,Actinotus helianthi,APC,https://id.biodiversity.org.au/node/apni/2895645,Labill.,Species,accepted,Apiaceae,"Qld, NSW, Vic (naturalised)",https://id.biodiversity.org.au/tree/51354547/5...,Actinotus


In [26]:
spp_info = CAPS[CAPS['scientificName'] == "Actinotus helianthi"] 
spp_info.transpose()

,3
speciesID,2361
taxonRank,Species
classID,38
class,Flora
orderID,137
order,Flora
familyID,157
family,Apiaceae
sortOrder,6031
genusID,1823.0


In [27]:
qry = target['acceptedNameUsageID'].values[0]+".json"
qry
#apniData=json.loads(qry) # error?

'https://id.biodiversity.org.au/node/apni/2895645.json'

In [28]:
apniData = json.loads(getResponse(qry))
apniData

{'treeElement': {'class': 'au.org.biodiversity.nsl.TreeElement',
  '_links': {'elementLink': 'https://id.biodiversity.org.au/tree/51662411/51242199',
   'taxonLink': 'https://id.biodiversity.org.au/node/apni/2895645',
   'parentElementLink': 'https://id.biodiversity.org.au/tree/51662411/51375672',
   'nameLink': 'https://id.biodiversity.org.au/name/apni/75081',
   'instanceLink': 'https://id.biodiversity.org.au/instance/apni/755469',
   'sourceElementLink': None},
  'tree': {'class': 'au.org.biodiversity.nsl.Tree',
   '_links': {'permalinks': [{'link': 'https://id.biodiversity.org.au/tree/apni/APC',
      'preferred': True,
      'resources': 1}]},
   'audit': None,
   'name': 'APC'},
  'simpleName': 'Actinotus helianthi',
  'namePath': 'Plantae/Charophyta/Equisetopsida/Magnoliidae/Asteranae/Apiales/Apiaceae/Actinotus/helianthi',
  'treePath': '/51209397/51209398/51209399/51210622/51236316/51241866/51242182/51375672/51242199',
  'displayHtml': '<data><scientific><name data-id=\'75081\'

## Explore traits

In [29]:
ATtraits['trait_name'].unique()

array(['leaf_area', 'leaf_delta13C', 'leaf_N_per_dry_mass', 'leaf_PRI',
       'modified_NDVI', 'specific_leaf_area', 'water_band_index',
       'wood_density', 'seed_length', 'seed_shape', 'seed_texture',
       'seed_width', 'seed_mass', 'germination_treatment',
       'dispersal_appendage', 'dispersal_syndrome',
       'leaf_dry_matter_content', 'plant_growth_form', 'plant_height',
       'leaf_length', 'leaf_thickness', 'leaf_width',
       'sapwood_specific_conductivity_theoretical', 'vessel_density',
       'vessel_diameter', 'vessel_lumen_fraction',
       'vessel_diameter_hydraulic', 'vessel_multiple_fraction',
       'water_use_efficiency_intrinsic', 'leaf_hairs_adult', 'leaf_shape',
       'fire_response', 'regen_strategy', 'fire_cued_seeding',
       'leaf_compoundness', 'seed_breadth', 'leaf_phenology',
       'flowering_time', 'life_history', 'leaf_type',
       'leaf_hydraulic_conductivity', 'leaf_hydraulic_vulnerability',
       'leaf_turgor_loss_point', 'ci_at_Amax', 'c

In [30]:
ss = (ATtraits['trait_name']=='resprouting_proportion_individuals' )
ATtraits[ss]
ss = (ATtraits['trait_name']=='fire_response' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
15332,Baker_2019,Commersonia bartramia,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_01,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Commersonia bartramia
15334,Baker_2019,Denhamia celastroides,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_02,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Denhamia celastroides
15336,Baker_2019,Blechnum neohollandicum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_03,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Doodia aspera
15338,Baker_2019,Pittosporum undulatum,Bogangar -28.3303611_and_153.5519444,NaN,Baker_2019_04,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Pittosporum undulatum
15340,Baker_2019,Polyscias sambucifolia,Main Arm -28.4936667_and_153.3923611,NaN,Baker_2019_05,fire_response,resprouts,NaN,NaN,expert_mean,NaN,Polyscias sambucifolia
...,...,...,...,...,...,...,...,...,...,...,...,...
959253,White_2020,Roepera ovata,NaN,NaN,White_2020_7923,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum ovatum
959272,White_2020,Roepera prismatotheca,NaN,NaN,White_2020_7924,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum prismatothecum
959291,White_2020,Roepera similis,NaN,NaN,White_2020_7925,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum simile
959310,White_2020,Roepera ammophila,NaN,NaN,White_2020_7926,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Zygophyllum sp. aff. ammophilum


In [31]:
ATtraits[ss]['value'].unique()

array(['resprouts', 'fire_killed', 'unknown', 'fire_killed resprouts',
       'not_fire_killed_does_not_resprout', 'fire_not_relevant',
       'fire_not_relevant resprouts',
       'fire_not_relevant not_fire_killed_does_not_resprout',
       'fire_killed fire_not_relevant'], dtype=object)

In [32]:
# ATtraits['trait_name'].unique()
ss = (ATtraits['trait_name']=='photosynthetic_pathway' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
175021,Cunningham_1999,Acacia binervata,Cunningham_Knights Hill_665m,NaN,Cunningham_1999_01,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia binervata
175037,Cunningham_1999,Acacia brachybotrya,Cunningham_Nombinnie_160m,NaN,Cunningham_1999_02,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia brachybotrya
175052,Cunningham_1999,Acacia rigens,Cunningham_Rankins-Springs_170m,NaN,Cunningham_1999_03,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia rigens
175068,Cunningham_1999,Acacia stricta,Cunningham_Narooma_25m,NaN,Cunningham_1999_04,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Acacia stricta
175083,Cunningham_1999,Boronia ledifolia,Cunningham_KNP-Waratah_165m,NaN,Cunningham_1999_05,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Boronia ledifolia
...,...,...,...,...,...,...,...,...,...,...,...,...
969325,Williams_2011,Austrostipa muelleri,NaN,grows in Adelaide,Williams_2011_947,photosynthetic_pathway,c4,NaN,NaN,expert_mean,NaN,Austrostipa muelleri
969334,Williams_2011,Deyeuxia minor,NaN,grows in Adelaide,Williams_2011_948,photosynthetic_pathway,c4,NaN,NaN,expert_mean,NaN,Deyeuxia minor
969360,Williams_2011,Aphanes australiana,NaN,grows in Adelaide,Williams_2011_951,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Aphanes australiana
969371,Williams_2011,Lomandra filiformis,NaN,grows in Adelaide,Williams_2011_952,photosynthetic_pathway,c3,NaN,NaN,expert_mean,NaN,Lomandra filiformis


### Dormancy type

In [33]:
ss = (ATtraits['trait_name']=='dormancy_type' )
ATtraits[ss]

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
479894,Ooi_2007,Acacia binervata,Fredericktown,NaN,Ooi_2007_00001,dormancy_type,physical_dormancy,NaN,1978-11-21,expert_mean,NaN,Acacia binervata
481466,Ooi_2007,Angophora bakeri,Agnes Banks to Castlereagh,NaN,Ooi_2007_01580,dormancy_type,non_dormant,NaN,1977-02-18,expert_mean,NaN,Angophora bakeri
488008,Ooi_2007,Isopogon anemonifolius,Cordeaux Cataract Catchment,NaN,Ooi_2007_08122,dormancy_type,physiological_dormancy,NaN,1975-08-12,expert_mean,NaN,Isopogon anemonifolius
489445,Ooi_2007,Bulbine bulbosa,Blacktown,NaN,Ooi_2007_09558,dormancy_type,morphophysiological_dormancy,NaN,1974-12-04,expert_mean,NaN,Bulbine bulbosa
489745,Ooi_2007,Calotis cuneifolia,Ashford,NaN,Ooi_2007_09858,dormancy_type,non_dormant physiological_dormancy,NaN,1978-11-25,expert_mean,NaN,Calotis cuneifolia


## Start XML file here

In [34]:
frdbCode='test'
frdbVersion='0.1'
frdbDate='2021-09-29'
sppname='Actinotus helianthi'


# write xml file
xml_dir = repodir / "xml"
if not os.path.isdir(xml_dir):
    os.makedirs(xml_dir)

file_name = xml_dir / sppname.replace(" ","_").replace(".","_").replace("/","_")
xml_file = file_name.with_suffix(".xml")

In [35]:
if os.path.isfile(xml_file):
    print('File exists')
else:
    root = ET.Element("SpeciesList")
    spp = ET.SubElement(root, "Species",code=frdbCode,version=frdbVersion,
                    update=frdbDate)
    ET.SubElement(spp, "Name").text = sppname
    ET.SubElement(spp, "Nomenclature")
    ET.SubElement(spp, "ImportedTraits")
    xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent=" ")
    with open(xml_file,"w") as f:
        f.write(xmlstr) #xmlstr.encode('utf-8')

File exists


In [36]:
tree = ET.parse(xml_file)
root = tree.getroot()

In [37]:
for spp in root.iter('Species'):
    impTraits = spp.find('ImportedTraits')

In [38]:
list(impTraits)
traitIds=[]
for trait in impTraits.iter('trait'):
    traitIds.append(trait.get('id'))
set(traitIds)
        

{'Falster_2005_2_05', 'NSWFRD_2014_0177'}

In [39]:
traits=('fire_response_juvenile','fire_response','fire_cued_seeding')
ss = (ATtraits['taxon_name']==sppname) & (ATtraits['trait_name'].isin(traits))
tgtTraits = ATtraits[ss]
tgtTraits

,dataset_id,taxon_name,site_name,context_name,observation_id,trait_name,value,unit,date,value_type,replicates,original_name
217795,Falster_2005_2,Actinotus helianthi,Myall_Lakes,NaN,Falster_2005_2_05,fire_response,fire_killed,NaN,2002-09,expert_mean,NaN,Actinotus helianthi
454654,NSWFRD_2014,Actinotus helianthi,NaN,NaN,NSWFRD_2014_0177,fire_response,fire_killed,NaN,NaN,expert_mean,NaN,Actinotus helianthi


Function to add traits based on _austraits_ trait table:

In [40]:
def addTraitNode(node,record):
    item=ET.SubElement(node,'trait',source='AusTraits',version='???',id=record['observation_id'],
                       name_used=record['original_name'])
    ET.SubElement(item,'name').text=record['trait_name'] 
    if ~np.isnan(record['unit']):
        ET.SubElement(item,'value',type=record['value_type'],unit=record['unit']).text=record['value']
    else:
        ET.SubElement(item,'value',type=record['value_type']).text=record['value'] 
    if ~np.isnan(record['replicates']):
        print('replicates')
    ET.SubElement(item,'dataset').text=record['dataset_id']
    if (isinstance(record['site_name'],str)):
        if record['site_name'] != '':
            ET.SubElement(item,'site').text=record['site_name'] 
    if (isinstance(record['date'],str)):
        if record['date'] != '':
            ET.SubElement(item,'date').text=record['date'] 

In [41]:
for index, row in tgtTraits.iterrows():
    if row['observation_id'] in set(traitIds):
        print('skipping existing trait record')
    else:
        addTraitNode(impTraits,row)

skipping existing trait record
skipping existing trait record


In [42]:
list(impTraits)


[<Element 'trait' at 0x119e27e20>, <Element 'trait' at 0x119e26890>]

In [43]:
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent=" ")

In [44]:
with open(xml_file,"w") as f:
    f.write(xmlstr) #xmlstr.encode('utf-8')